## Stammzelltisch 9.4.2024 - Large language model tutorial

## Background
* There are a couple of different large language models available
    * OpenAI models
        * GPT-3.5
        * GPT-4.0
    * LLama2
        * 7, 13, 70 billion parameters
        * Here 7 billion parameters, 4-bits --> can be run on laptop with 8 GBs
    * Gemma
    * ...

### Initialize large language model

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
# llm = Ollama(model="gemma:2b")

### Query large language model

In [8]:
print(llm.invoke("Please show me a recipe for a vegan Tiramisu!"))


Yay, a vegan Tiramisu! This classic Italian dessert is now easily accessible to you with these simple and delicious vegan ingredients. Here's a recipe that will satisfy your cravings:

Ingredients:

For the ladyfingers:

* 12-16 ladyfingers (you can use any type of cookie or crackers, but ladyfingers are traditional)
* 3/4 cup granulated sugar
* 1/4 cup vegan coffee liqueur (such as Kahlúa or Tia Maria)
* 1/4 cup non-dairy milk (such as almond, soy, or coconut milk)

For the filling:

* 1/2 cup cashew cream (see recipe below)
* 1/4 cup vegan chocolate chips (such as Enjoy Life or DaVinci)
* 1/4 cup granulated sugar
* 1 tsp vanilla extract

Cashew Cream:

* 1/2 cup raw cashews
* 1/4 cup non-dairy milk (such as almond, soy, or coconut milk)
* 1 tsp lemon juice
* 1/4 tsp salt

Instructions:

1. In a small bowl, mix together the sugar and coffee liqueur for the ladyfingers.
2. Dip each ladyfinger into the sugar and coffee mixture, coating both sides evenly. Place on a serving plate or tra

### Modify the behavior by modifying the prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You talk like an Australian guy."),
    ("user", "{input}")
])

chain = prompt | llm 
print(chain.invoke({"input": "What is life all about?"}))

### Test what the llm knows about stem cell biology

In [9]:
print(llm.invoke("Please explain the concept of stemness of stem cells to me!"))


Stemness is a term used to describe the properties and behaviors of stem cells, which are a type of cell that has the ability to differentiate into different cell types in the body. Stem cells are found in various tissues throughout the body, including bone marrow, blood, and the placenta during fetal development.

The concept of stemness refers to the unique properties of stem cells that allow them to self-renew ( divide and produce more stem cells) and differentiate into different cell types. These properties include:

1. Self-renewal: Stem cells have the ability to divide and produce more stem cells, allowing them to maintain their population in the body.
2. Multipotency: Stem cells can differentiate into multiple cell types, such as blood cells, nerve cells, and muscle cells.
3. Plasticity: Stem cells can change their fate and differentiate into different cell types depending on the environmental cues they receive.
4. Self-repair: Stem cells have the ability to repair damaged tiss

In [10]:
print(llm.invoke("Please explain the concept of within-tissue plasticity to me!"))


Within-tissue plasticity refers to the ability of cells within a tissue or organ to change their position, shape, and function in response to changes in the local microenvironment. This concept is important in understanding how tissues adapt to injury or disease, and how they regenerate after an insult.

In a healthy tissue, cells are arranged in a specific spatial arrangement, with each cell type performing a unique set of functions to maintain tissue homeostasis. However, when a tissue is injured or diseased, the normal structure and function of the tissue can be disrupted. Within-tissue plasticity allows cells within the tissue to adapt to these changes by rearranging themselves and altering their gene expression profiles.

For example, in response to injury, fibroblasts in a tissue may undergo epithelial-to-mesenchymal transition (EMT), a process where they lose their epithelial characteristics and acquire mesenchymal properties, such as increased mobility and the ability to produ

## Use retrieval augmented generation

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/12160836/")
docs = loader.load()

In [12]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Please explain the concept of stemness to me!"})
print(response["answer"])



Stemness is a term used in cell biology to describe the state of a cell that has the ability to differentiate into multiple cell types. In other words, stem cells are cells that have the potential to become any type of cell in the body. This property is due to the presence of specific genetic and epigenetic markers that allow stem cells to maintain their undifferentiated state and self-renew while also giving rise to differentiated cells.

Stemness is a complex phenomenon that involves a balance between different molecular pathways and cellular processes, including cell adhesion, signaling, and transcriptional regulation. Stem cells are able to maintain their stemness by regulating the expression of genes involved in self-renewal and differentiation, as well as by modulating their microenvironment through secreted factors and cell-cell interactions.

There are two main types of stem cells: embryonic stem cells (ESCs) and adult stem cells (ADSCs). ESCs are derived from embryos and have

## Use RAG for getting information about a specific article

In [ ]:
result = llm.invoke("What did Lutz Leichsenring tell the German broadcaster DW?")
print(result)


In [ ]:
loader = WebBaseLoader("https://www.theguardian.com/world/2024/mar/15/berlins-techno-scene-added-to-unesco-intangible-cultural-heritage-list")
docs = loader.load()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "What did Lutz Leichsenring tell the German broadcaster DW?"})
print(response["answer"])
